In [ ]:
!pip install pyspark

In [3]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("ModelTraining").config("spark.mongodb.input.uri", "mongodb://localhost:27017/youtube_db.commentvideo").getOrCreate()

df = spark.read.format("com.mongodb.spark.sql.DefaultSource").load()
df.show()

ModuleNotFoundError: No module named 'pyspark'